In [1]:
# Libraries
import pandas as pd
pd.set_option('display.max_columns', 40)
pd.set_option('display.width', 2000)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
%pip install \
--trusted-host 192.168.210.244 \
--index-url http://192.168.210.244:8081/repository/pypi/simple/ \
snowflake-connector-python

In [3]:
import json, snowflake.connector

# establish the connection to snowflake
ctx = snowflake.connector.connect( 
    **json.load(open('/opt/ich/python-snowflake-defaults.json')))
    
# verify and test if connection is working
try: 
    cs = ctx.cursor() 
    cs.execute('SELECT current_version(), current_role(), current_warehouse()')
    print(cs.fetchone())
finally: 
    cs.close()

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/snowflake/connector/options.py:96: UserWarning: You have an incompatible version of 'pyarrow' installed (6.0.0), please install a version that adheres to: 'pyarrow<8.1.0,>=8.0.0; extra == "pandas"'
  warn_incompatible_dep(


Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
We were unable to open a browser window for you, please open the following url manually then paste the URL you are redirected to into the terminal.
URL: https://login.microsoftonline.com/be00e2c6-806c-45f8-84c3-3fc99f64b8d3/saml2?SAMLRequest=pZPNctowFIVfxaOuLcs%2FZbAGkyGhpDRpygTSBTshX4MGW3IlGTt5%2BsoGZtJFsulOI52r79x7pMlNV5XeCbQRSmYoxAR5ILnKhdxn6GWz8MfIM5bJnJVKQoZewaCb6cSwqqzprLEH%2BQx%2FGjDWcxdJQ%2FuDDDVaUsWMMFSyCgy1nK5nPx9phAllxoC2DocuJbkRjnWwtqZB0LYtbmOs9D6ICCEBSQOn6iVf0DtE%2FTmj1soqrsprSed6%2BgARBiTpEU7hCKtL4a2Q5xF8RtmdRYZ%2B32xW%2FurXeoO82bW7OyVNU4Fegz4JDi%2FPj2cDxjmodZKmJMJSaXvwodGqBlxrcWIWSiGP2EjVFiU7AldV3VgHwW4VFJAHpdoLN7rlPEP1UeRpuO1GyY%2B3HbMPT6d4O%2BtsMp8f0lh169vRw0J03%2B5Xy939qSUceb%2BvQUd90EtjGljKPl7rtkgU%2ByTyo3ATxjSOaEJwQsIt8uYuXiGZHSqvPQw%2B

('7.6.1', 'ICHT_RG_COVOAM_22016_PROD', 'ICHT_WH_COVOAM_22016_PROD')


True

In [3]:
# Import
path = r'data/drugs_indications_and_codes.csv'
drugs_indications_and_codes = pd.read_csv(path)

path = r'data/final_problem_dummies.csv'
final_problem_dummies = pd.read_csv(path)

In [4]:
# Import 
query = '''
SELECT * from ICHT_PROD.ICHT_COVID.PHARMACY_PRESCRIBING
'''
cur = ctx.cursor().execute(query)
prescribing_df = pd.DataFrame.from_records(iter(cur), columns=[x[0] for x in cur.description])

In [5]:
# Filter for administration
prescribing_df = prescribing_df[prescribing_df['ADMISSION_MEDICINE_Y_N'] == 'Yes']
prescribing_df = prescribing_df[['SUBJECT', 'PRESCRIPTION_ORDER_ID','ORDER_DT_TM', 'PRESCRIPTION_TYPE', 'THERAPEUTICAL_CLASS', 'MEDICATION_NAME_SHORT', 'MEDICATION_NAME']]

In [7]:
# Filter for those subjects of interest
prescribing_df = prescribing_df[prescribing_df['SUBJECT'].isin(final_problem_dummies['SUBJECT'].to_list())]

In [9]:
# Filter for medications of interest
prescribing_df = prescribing_df[prescribing_df['MEDICATION_NAME_SHORT'].isin(drugs_indications_and_codes['icht_prescribing_medication'].to_list())]

In [11]:
# Save
#prescribing_df.to_csv('filtered_pharmacy_prescribing.csv')

In [17]:
# Convert to datetime
final_problem_dummies['PROBLEM_DT_TM'] = pd.to_datetime(final_problem_dummies['PROBLEM_DT_TM'])
prescribing_df['ORDER_DT_TM'] = pd.to_datetime(prescribing_df['ORDER_DT_TM'])

In [ ]:
for x in range(len(final_problem_dummies)):
    if x % 1000 == 0:
        print('x:', x)
    # Filter by subject
    temp_df = prescribing_df[prescribing_df['SUBJECT'] == final_problem_dummies.loc[x, 'SUBJECT']]
    if len(temp_df) >= 1:
        # Filter by date
        temp_df = temp_df[temp_df['ORDER_DT_TM'] <= final_problem_dummies.loc[x, 'PROBLEM_DT_TM']]
        if len(temp_df) >= 1:
            temp_drug_df = drugs_indications_and_codes[drugs_indications_and_codes['icht_prescribing_medication'].isin(temp_df['MEDICATION_NAME_SHORT'].to_list())]
            snomed_code_list = temp_drug_df['snomed_code'].astype(str).unique().tolist()
            # Add prefix 
            prefix_ADD = 'PROBLEM_'
            snomed_code_list = [prefix_ADD + z for z in snomed_code_list if isinstance(z, str)]
            # Find snomed codes that overlap
            snomed_code_list = list(set(snomed_code_list) & set(final_problem_dummies.iloc[:,3:].columns.tolist()))
            # Set those problems to 1 in main df
            final_problem_dummies.loc[x, snomed_code_list] = 1

In [20]:
# Save
#final_problem_dummies.to_csv('final_problem_dummies_med.csv')

## Run with fuzzy

In [22]:
# Import
path = r'data/drugs_indications_and_codes_fuzzy.csv'
drugs_indications_and_codes_fuzzy = pd.read_csv(path)

In [23]:
# Import
path = r'data/final_problem_dummies.csv'
final_problem_dummies = pd.read_csv(path)

In [26]:
# Convert to datetime
final_problem_dummies['PROBLEM_DT_TM'] = pd.to_datetime(final_problem_dummies['PROBLEM_DT_TM'])
prescribing_df['ORDER_DT_TM'] = pd.to_datetime(prescribing_df['ORDER_DT_TM'])

In [ ]:
for x in range(len(final_problem_dummies)):
    if x % 1000 == 0:
        print('x:', x)
    # Filter by subject
    temp_df = prescribing_df[prescribing_df['SUBJECT'] == final_problem_dummies.loc[x, 'SUBJECT']]
    if len(temp_df) >= 1:
        # Filter by date
        temp_df = temp_df[temp_df['ORDER_DT_TM'] <= final_problem_dummies.loc[x, 'PROBLEM_DT_TM']]
        if len(temp_df) >= 1:
            temp_drug_df = drugs_indications_and_codes_fuzzy[drugs_indications_and_codes_fuzzy['icht_prescribing_medication'].isin(temp_df['MEDICATION_NAME_SHORT'].to_list())]
            snomed_code_list = temp_drug_df['snomed_code'].astype(str).unique().tolist()
            # Add prefix 
            prefix_ADD = 'PROBLEM_'
            snomed_code_list = [prefix_ADD + z for z in snomed_code_list if isinstance(z, str)]
            # Find snomed codes that overlap
            snomed_code_list = list(set(snomed_code_list) & set(final_problem_dummies.iloc[:,3:].columns.tolist()))
            # Set those problems to 1 in main df
            final_problem_dummies.loc[x, snomed_code_list] = 1

In [29]:
# Save
#final_problem_dummies.to_csv('final_problem_dummies_med_fuzzy.csv')